In [25]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import anthropic
from google import genai

load_dotenv()
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

### Using local models - ollama
Run `ollama run deepseek-r1:8b` locally on terminal before doing 

Visiting `http://localhost:11434` should give `Ollama is running`, if not then `ollama serve`

In [5]:
MODEL = "deepseek-r1:8b"
ollama_model = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

response = ollama_model.chat.completions.create(
 model=MODEL,
 messages=[{"role": "user", "content": "who are you ?"}]
)

print(response.choices[0].message.content)

<think>

</think>

Greetings! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. I'm at your service and would be delighted to assist you with any inquiries or tasks you may have.


### OpenAI model

In [7]:
openai = OpenAI()
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(
    model="o4-mini",
    messages=[{"role":"user", "content":message}]
)
print(response.choices[0].message.content)

Hello there! Welcome, and thanks for saying hi. I’m ChatGPT—here to help answer questions, brainstorm ideas, or just chat. What can I do for you today?


### AI web summarizer

In [28]:
# some websites need a proper header when fetching them, below header is copied from dev tools under request headers
# BeautifulSoup is ideal for static pages and simple scraping tasks, offering speed and efficiency. Selenium is better for dynamic content or tasks requiring user interaction, like form submission or clicking buttons.

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        """
        Create this Website object from the given url using BeautifulSoupe Library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelavent in soup.body(["script", "style", "img", "input"]):
            irrelavent.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        self.soup = soup


In [32]:
brk = Website("https://www.linkedin.com/in/bharath-k-reddy/")
print(brk.title)
print(brk.text)

Bharath Reddy - Northern Trust | LinkedIn
LinkedIn and 3rd parties use essential and non-essential cookies to provide, secure, analyze and improve our Services, and to show you relevant ads (including
professional and job ads
) on and off LinkedIn. Learn more in our
Cookie Policy
.
Select Accept to consent or Reject to decline non-essential cookies for this use. You can update your choices at any time in your
settings
.
Accept
Reject
Skip to main content
LinkedIn
Top Content
People
Learning
Jobs
Games
Get the app
Join now
Sign in
Sign in to view Bharath’s full profile
Sign in
Welcome back
Email or phone
Password
Show
Forgot password?
Sign in
or
By clicking Continue to join or sign in, you agree to LinkedIn’s
User Agreement
,
Privacy Policy
, and
Cookie Policy
.
New to LinkedIn?
Join now
or
New to LinkedIn?
Join now
By clicking Continue to join or sign in, you agree to LinkedIn’s
User Agreement
,
Privacy Policy
, and
Cookie Policy
.
Bharath Reddy
Sign in to view Bharath’s full profile
S

In [42]:
system_prompt = "You are an assistant that analyses the contents of a website \
    and provides a short summary, ignoring text that might be navigation related. \
    Respond in markdown."

In [43]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled: {website.title}"
    user_prompt += "The contents of this website is as follows; \
        please provide a short summary of this website in markdown.\
        if it includes news or announcements, then summarize these too. \n\n"
    user_prompt += website.text
    return user_prompt

In [44]:
blog = Website("https://www.anthropic.com/engineering/built-multi-agent-research-system?utm_source=alphasignal&utm_campaign=2025-07-18&asuniq=02f5e591")
user_prompt_for(blog)

"You are looking at a website titled: How we built our multi-agent research system \\ AnthropicThe contents of this website is as follows;         please provide a short summary of this website in markdown.        if it includes news or announcements, then summarize these too. \n\nSkip to main content\nSkip to footer\nClaude\nAPI\nSolutions\nResearch\nCommitments\nLearn\nNews\nTry Claude\nEngineering at Anthropic\nHow we built our multi-agent research system\nPublished\nJun 13, 2025\nOur Research feature uses multiple Claude agents to explore complex topics more effectively. We share the engineering challenges and the lessons we learned from building this system.\nClaude now has\nResearch capabilities\nthat allow it to search across the web, Google Workspace, and any integrations to accomplish complex tasks.\nThe journey of this multi-agent system from prototype to production taught us critical lessons about system architecture, tool design, and prompt engineering. A multi-agent system

In [45]:
def messages_for(website):
    return[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

messages_for(blog)

[{'role': 'system',
  'content': 'You are an assistant that analyses the contents of a website     and provides a short summary, ignoring text that might be navigation related.     Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled: How we built our multi-agent research system \\ AnthropicThe contents of this website is as follows;         please provide a short summary of this website in markdown.        if it includes news or announcements, then summarize these too. \n\nSkip to main content\nSkip to footer\nClaude\nAPI\nSolutions\nResearch\nCommitments\nLearn\nNews\nTry Claude\nEngineering at Anthropic\nHow we built our multi-agent research system\nPublished\nJun 13, 2025\nOur Research feature uses multiple Claude agents to explore complex topics more effectively. We share the engineering challenges and the lessons we learned from building this system.\nClaude now has\nResearch capabilities\nthat allow it to search across the web, Google Works

In [52]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages=messages_for(website)
    )
    return response.choices[0].message.content

def dispaly_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

dispaly_summary("https://www.anthropic.com/engineering/built-multi-agent-research-system?utm_source=alphasignal&utm_campaign=2025-07-18&asuniq=02f5e591")

# Summary of "How We Built Our Multi-Agent Research System"

The article details the engineering journey of developing a multi-agent research system using multiple Claude language models (LLMs) to tackle complex research tasks more efficiently. This system allows the lead agent to orchestrate several subagents that operate concurrently, which increases performance, especially for dynamic and unpredictable research queries.

## Key Highlights

- **Multi-Agent System Overview**: The system employs a lead agent that plans the research process and spawns various subagents to work in parallel, improving problem-solving capabilities by allowing broader exploration without being path-dependent.

- **Research Efficiency**: Internal evaluations showed that the multi-agent system significantly outperformed single-agent setups, particularly in tasks requiring extensive information retrieval.

- **Token Usage and Performance**: A multi-agent system utilizes token calls more effectively, substantially improving efficiency in complex queries, although it requires managing higher token costs for economic feasibility.

- **Architecture Description**: The architecture involves an orchestrator-worker model, where the lead agent creates subagents for specific tasks, showcasing a dynamic approach to information gathering compared to traditional static methods.

- **Prompt Engineering**: Essential principles for effective prompting were discussed, emphasizing the importance of clear delegation and task separation for achieving optimal performance from the agents.

- **Production Challenges**: The article explores various challenges faced in deploying and maintaining such systems, including error handling, state management, and the coordination of asynchronous operations.

- **Human Evaluation**: While automated evaluations assist in measuring performance, human testers play a critical role in identifying edge cases and potential biases in agent behavior.

## Usage Insights

Anthropic's research feature is utilized across various domains, such as software development, content optimization, business strategy, and academic research, reflecting its versatility in handling complex, open-ended tasks.

## Conclusion

The engineering process highlighted the gap between prototype and production, emphasizing the extensive work needed to create reliable multi-agent systems capable of significant problem-solving in research contexts. Regular feedback loops and collaborative efforts are central to refining these systems for scaling and reliability.

This article was published on June 13, 2025, and reflects contributions from multiple team members at Anthropic.

### selenium web scapping for sites which use js

In [ ]:
# needs chrome webdriver (https://sites.google.com/chromium.org/driver/)
import undetected_chromedriver as uc
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By # select elements
from selenium.webdriver.common.keys import Keys
import time
# service = Service(executable_path="./chromedriver")
# driver = webdriver.Chrome(service=service) 

driver = uc.Chrome(headless=False)

driver.get("https://openai.com/index/introducing-chatgpt-agent/")
time.sleep(5)
driver.quit()

In [ ]:
import undetected_chromedriver as uc
from openai import OpenAI

class BRK_Scaper:
    def __init__(self, url, model_provider, model_version="deepseek-r1:8b", ):
        """
        Create this Website object from the given url using BeautifulSoupe Library
        """
        self.url = url
        self.driver = uc.Chrome(headless=False)
        self.model_version = model_version
        self.model_provider = model_provider

    def get_page(self):
        self.driver.get(self.url)
        html = self.driver.page_source
        self.driver.quit()
        soup = BeautifulSoup(html, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelavent in soup.body(["script", "style", "img", "input", "nav", "style", ]):
            irrelavent.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)
        # self.soup = soup (for debugging or selectively retaining page contents)

    
        
    def get_summary(self):
        
        if self.model_provider=='ollama':
            llm = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
        elif self.model_provider=='openai':
            llm = OpenAI()
        else:
            raise ValueError(f"Unknown model provider: {self.model_provider}")

        self.system_prompt = "You are an assistant that analyses the contents of a website \
            and provides a short summary, ignoring text that might be navigation related. \
            Respond in markdown."
        
        self.human_prompt = f"You are looking at a website titled: {self.title} \
            please provide a short summary of this website in markdown.\
            if it includes news or announcements, then summarize these too. \n\n \
            {self.text}" 

        response = llm.chat.completions.create(
            model=self.model_version,
            messages=[
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.human_prompt}
            ]
        )

        self.summary = response.choices[0].message.content
        display(Markdown(self.summary))
        

In [80]:
url = "https://openai.com/index/introducing-chatgpt-agent/"
scrapper = BRK_Scaper(url=url, model_provider='openai', model_version="gpt-4o-mini")
scrapper.get_page()
scrapper.get_summary()

# Summary of "Introducing ChatGPT agent: bridging research and action | OpenAI"

On July 17, 2025, OpenAI announced the launch of the **ChatGPT agent**, a new feature that enhances ChatGPT's capabilities to think and act proactively, utilizing its own computer to perform complex tasks. Users can now delegate tasks such as analyzing calendars, planning meals, or creating presentations, with ChatGPT intelligently navigating the web and handling workflows autonomously. 

### Key Features:
- **Unified Agentic System**: Combines strengths from previous models (Operator for web interaction and deep research for information synthesis) to perform tasks efficiently.
- **Tool Suite**: Includes a visual browser and text-based browser for web interactions, terminal access, and API integration.
- **Iterative Workflows**: Users can interrupt tasks and provide additional directions as needed, allowing for a more collaborative experience.
- **Real-World Utility**: Useful for both personal and professional tasks, such as scheduling appointments, planning travel, or automating reports.
- **Safety Measures**: Enhanced controls for data privacy and task confirmations to reduce risks associated with accessing live web data.

### Performance Highlights:
- Achieved a state-of-the-art performance on various evaluation benchmarks, such as completing complex real-world tasks and improving on previous models.
  
### Availability:
The ChatGPT agent starts rolling out to Pro, Plus, and Team users, with access expanding to other user tiers in the coming weeks.

### Risks and Controls:
OpenAI outlined potential risks, particularly regarding data usage and web interactions, implementing action confirmations and user oversight to mitigate these concerns. 

### Future Enhancements:
OpenAI plans to continuously improve the ChatGPT agent's functionality, expanding its capabilities while ensuring user safety.

This launch marks the beginning of an evolving interaction model, where users can expect enhanced intelligence and automation in handling tasks.

### Streaming in jupiter  
##### Code-copilot

In [16]:
from openai import OpenAI
client = OpenAI()

code_to_explain = 'yield{book.get("author") for book in books if book.get("author")}'
system_prompt = "You are a helpful AI agent who explains given code in clear markdown format."
user_prompt = f"Explain below code to me. Provide your responce in short and clear markdown format. \n\n code \n\n {code_to_explain}"


completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
  ],
  stream=True
)

for chunk in completion:
  print(chunk.choices[0].delta.content, end=' ')


 ###  Explanation  of  the  Code 

 The  given  code  snippet  is  using  a  generator  expression  to  yield  values .  Here's  a  breakdown :

 1 .  ** Generator  Expression ** :  
     -  The  code  is  creating  a  generator  that  will  yield  a  set  of  unique  authors .

 2 .  ** Set  Com preh ension ** :  
     -  The  syntax  `{ ...  for  ...  in  ... }`  indicates  that  a  set  is  being  constructed .
     -  This  set  will  contain  values  returned  by  ` book .get (" author ") `.

 3 .  ** Iteration ** :
     -  It  iter ates  over  ` books `,  which  is  presumably  a  list  or  collection  of  ` book `  dictionaries .

 4 .  ** Condition ** :
     -  The  ` if  book .get (" author ") `  condition  filters  out  any  ` book `  entries  that  do  not  have  an  `" author "`  key  or  where  the  value  is  ` None `  ( or  evaluates  to  False ).

 ###  Summary 

 This  code  yields  a  set  of  authors  from  a  collection  of  book  dictionaries ,  including  only  th

In [21]:
client = anthropic.Anthropic()

message = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=200,
    temperature=0.7,
    system="You are a world-class poet. Respond only with short poems.", #sys msg is seperate attribute
    messages=[
        {"role": "user", "content": "Why is the ocean salty?"}
    ]
)
print(message.content[0].text)

Waves crash, salt disperses,
Oceans hold the world's verses.
Tides ebb and flow, ever-changing,
Saltiness, a story ever-engaging.


In [22]:
result = client.messages.stream(
    model="claude-3-haiku-20240307",
    max_tokens=200,
    temperature=0.7,
    system="You are a world-class poet. Respond only with short poems.", #sys msg is seperate attribute
    messages=[
        {"role": "user", "content": "Why is the ocean salty?"}
    ]
)
with result as stream:
    for text in stream.text_stream:
        print(text, end="", flush=True)


Waves crashing, salt dispersing,
Oceans vast, their depths conversing.
Minerals leached, from land to sea,
Saltiness, nature's mystery.

In [28]:
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash-lite-preview-06-17",
    contents="Explain how AI works in a few words",
    # config=genai.types.GenerateContentConfig(
    #     thinking_config=genai.types.ThinkingConfig(thinking_budget=0) # Disables thinking,otherwise on by default
    # ),
)

print(response.text)

AI learns from data to make predictions or decisions.


### Adversarial chat between `Claude` and `openai`

In [54]:
class OpenaiModel:
    user_task: str

    def __init__(self, user_task):
        self.system_message = "You are a world class fullstack web developer. You carefully plan your development steps \
            and good at refactoring your code based on feedback."
        self.human_message = user_task
        self.messages=[
            {"role": "system", "content": self.system_message},
            {"role": "user", "content": self.human_message}
        ]
        self.responses=[]

    def generate_response(self):
        client = OpenAI()
        response = client.chat.completions.create(
            model="o4-mini",
            messages=self.messages
        )
        self.responses.append(response.choices[0].message.content)
        return response.choices[0].message.content

    def incorporate_feedback(self, feedback):
        self.messages.append({"role": "assistant", "content": self.responses[-1]})  # previous code
        self.messages.append({"role": "user", "content": f"Here is the review feedback:\n{feedback}\n\
                              Please revise the code accordingly."})

class ClaudeModel:
    user_task:str

    def __init__(self):
        self.client = anthropic.Anthropic()
        self.system_message = "You are a senior engineer and you do code reviews. \
            You analyse codebase for design principles and code correctness to provide feedback on the code. \
            You would be given code and your job is to analyse the provided code and provide feedback."
        # self.human_message = f"Please review this code and provide your feedback. \n\n code:\n{user_task}"
        # self.messages=[
        #     {"role": "user", "content": self.human_message}
        # ]
        # self.responces=[]

    def generate_feedback(self, code):
        client = anthropic.Anthropic()
        response = client.messages.create(
            model="claude-3-5-haiku-latest",
            max_tokens=8192,
            temperature=0.7,
            system=self.system_message,
            messages=[
                {"role": "user", "content": f"Please review this code and provide your feedback:\n\n{code}"}
            ]
        )
    
        return response.content[0].text

In [ ]:
user_task = "Write a full functional single page web application to play human vs computer \
    tic tac. Should be just HTML, CSS and JS. Should track computer and human scores. \
        should allow human to reset the game. Computer and human should take turns to strat the game \
            with human starting first in best of 5 game match."

# Initialize models
openai_model = OpenaiModel(user_task)
claude_model = ClaudeModel()

# Feedback loop
for i in range(3):  
    print(f"\n--- Iteration {i+1} ---")
    code = openai_model.generate_response()
    print(f"✅ OpenAI's code:\n{code}\n\n....................")  # Truncated for readability

    feedback = claude_model.generate_feedback(code)
    print(f"🧐 Claude's feedback:\n\n{feedback}\n\n................")  # Truncated for readability

    openai_model.incorporate_feedback(feedback)




--- Iteration 1 ---
✅ OpenAI's code:
Here’s a single‐page, pure HTML/CSS/JS tic-tac-toe app. It tracks a best-of-5 match, alternates who starts each round (human starts round 1), keeps human/computer scores, and offers “Next Round” and “Reset Match” controls. The computer uses minimax for perfect play.

Save this as `index.html` and op

....................
🧐 Claude's feedback:

Here's a comprehensive code review of the Tic-Tac-Toe implementation:

### Strengths of the Implementation

1. **Modular and Self-Contained Design**
   - Single HTML file with embedded CSS and JavaScript
   - Immediately Invoked Function Expression (IIFE) for encapsulation
   - Clean separation of c

................

--- Iteration 2 ---
✅ OpenAI's code:
Below is a refactored single-page Tic-Tac-Toe app incorporating the review feedback:

Key improvements  
• CONFIG & GameState enums for easy tuning   
• Alpha-beta-pruned minimax with depth to prefer quick wins   
• More comments and strict const/let usage   


In [56]:
openai_model.messages

[{'role': 'system',
  'content': 'You are a world class fullstack web developer. You carefully plan your development steps             and good at refactoring your code based on feedback.'},
 {'role': 'user',
  'content': 'Write a full functional single page web application to play human vs computer     tic tac. Should be just HTML, CSS and JS. Should track computer and human scores.         should allow human to reset the game. Computer and human should take turns to strat the game             with human starting first in best of 5 game match.'},
 {'role': 'assistant',
  'content': 'Here’s a single‐page, pure HTML/CSS/JS tic-tac-toe app. It tracks a best-of-5 match, alternates who starts each round (human starts round 1), keeps human/computer scores, and offers “Next Round” and “Reset Match” controls. The computer uses minimax for perfect play.\n\nSave this as `index.html` and open in your browser.\n\n```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n  <meta charset="UTF-8">\n  <tit